## Version Description
In this version, we deploy our prototype Hybrid PINN with fixed learning rate to predict only one correction factor for the `MTR-600` in TZ6

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
# import required libraries
import pandas as pd
import numpy as np


#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.io import export_svgs
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

### Data Acquisition

In [4]:
target_df = pd.read_csv('../data/TZ6_dataset.csv')
target_df.head()

,HoV,CAOLH_SumFlow,CAORH_SumFlow,LAOLH_SumFlow,LAORH_SumFlow,TZ6_Flow,MIXT,AMBT,AMBP,MIXP,...,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R620_HS1,R620_HS2,R620_HS3,R621_HS1,R621_HS2,R621_HS3
0,A1,217.197005,222.443249,133.373448,136.693697,709.707399,293.150000,299.386667,101401.6000,2600.0000,...,131,136,120,120,75,38,113,75,38,113
1,A1,217.248929,223.698078,132.780168,135.684064,709.411240,293.150000,298.448667,101576.3000,2600.0000,...,131,136,120,120,75,38,113,75,38,113
2,A2,229.778639,226.831740,140.650844,143.718662,740.979885,298.101330,297.109024,102136.6035,2606.1928,...,131,136,114,120,75,38,113,75,38,113
3,A3,225.091032,227.788705,137.109226,142.122616,732.111579,294.390392,295.060027,103195.6642,2599.8998,...,131,136,120,120,75,38,113,75,38,113
4,A4,225.200989,222.557674,130.494552,135.055208,713.308423,293.150000,294.755833,102856.2000,2600.0000,...,145,153,130,130,75,38,113,75,38,113


In [5]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 21 columns):
HoV              35 non-null object
CAOLH_SumFlow    35 non-null float64
CAORH_SumFlow    35 non-null float64
LAOLH_SumFlow    35 non-null float64
LAORH_SumFlow    35 non-null float64
TZ6_Flow         35 non-null float64
MIXT             35 non-null float64
AMBT             35 non-null float64
AMBP             35 non-null float64
MIXP             35 non-null float64
R600_HD          35 non-null int64
R610_HS1         35 non-null int64
R611_HS1         35 non-null int64
R612_HS1         35 non-null int64
R613_HS1         35 non-null int64
R620_HS1         35 non-null int64
R620_HS2         35 non-null int64
R620_HS3         35 non-null int64
R621_HS1         35 non-null int64
R621_HS2         35 non-null int64
R621_HS3         35 non-null int64
dtypes: float64(9), int64(11), object(1)
memory usage: 5.8+ KB


In [6]:
# Define Restrictors and their parameters
MTR = 'R600_HD'

# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m

## Define Initial Correction factor values from CFD
MTR_cf = 1.1664876228437

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
_,target_df[MTR+'_Zeta1D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Zeta1D']].head()

,R600_HD,R600_HD_Zeta1D
0,148,4.949375
1,149,4.730821
2,152,4.127180
3,154,3.764504
4,148,4.949375


In [9]:
# Gather all column names required for the input loss coefficients
zeta_col_names = [col for col in target_df.columns if 'Zeta1D' in col]

### Training data Preparation

In [10]:
from TZ6_1CF import NormbyMax, PINN

In [11]:
input_features = ['R600_HD_Zeta1D']
features_max, df_rescaled = NormbyMax(target_df,input_features)
print('Max value of all features:',features_max)

V_max_org = target_df['TZ6_Flow'].max()
print('Maximum TZ6 Flow:',V_max_org)

Max value of all features: {'R600_HD_Zeta1D': 5.4148802645163}
Maximum TZ6 Flow: 764.0123719999998


In [12]:
print("Number of training samples:", len(df_rescaled))
# Display original and Scaled DataFrame Side-by-side
print( '\n{:<20s} {:<20s}'.format('Original Input Dataframe:','Rescaled Input Dataframe:') )
display_side_by_side(target_df[['HoV']+input_features],df_rescaled)

Number of training samples: 35

Original Input Dataframe: Rescaled Input Dataframe:


,HoV,R600_HD_Zeta1D
0,A1,4.949375
1,A1,4.730821
2,A2,4.127180
3,A3,3.764504
4,A4,4.949375
5,A5,4.949375
6,C1,4.949375
7,C2,3.942093
8,C3,4.127180
9,C4,4.521189


In [13]:
# Store indices and values of train data
train_data = df_rescaled.iloc[[11]]#.iloc[:5]
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = target_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [14]:
### Set the hyperparameters here ###
learning_rate =  0.01 ### Use 0.1 for CF-only loss, for all else 0.01
MTR_epsi = np.float(MTR_cf * 0.01)

N_i = train_data.shape[1] # No. of input feature columns in the dataframe
hl_dim = [1] ### Important Parameter
output_nodes = 1
network_dim = (N_i, hl_dim, output_nodes)

#### Train data points sequentially - Single Data Point Training Mode

In [15]:
%%time
## initialize hybrid PINN network
network = PINN(MTR_epsi, learning_rate, activation_function_choice = 'linear', error_loss_choice = 'simplified') #full, simplified, cf_only
MSE_flowloss_hist_train, error_loss_hist_train, cf_hist_train, losses_hist = network.train_1p_seq(network_dim, input_features, train_data[input_features], train_R600HD_series, train_data_idx, MTR_DuctArea, target_df, zeta_col_names, V_max_org) 

Activation Function selected: linear
Loss Function & Delta-k Used: simplified
Fixed Learning_rate Used: 0.01

INITIAL WEIGHTS & BIASES:
W1: [array([0.497])] <class 'numpy.ndarray'> Shape: (1, 1)
b1: [1.0] <class 'numpy.ndarray'> Shape: (1,)
W2: [array([-0.138])] <class 'numpy.ndarray'> Shape: (1, 1)
b2: [1.0] <class 'numpy.ndarray'> Shape: (1,)

BEGIN Neural Network training for ['C6']

MTR-Cf: 0.8139898685203247
Row ID: 11 	HoV: ['C6']
FlowRate Difference (LTR - FDDN): -54.372492943537054 l/s
EPOCH: 1 ;	Progress: 2.0% ;	MSE Flowloss (Train):  2956.368 ;	Error_loss (Train):  1.0129

MTR-Cf: 0.9827040594133117
Row ID: 11 	HoV: ['C6']
FlowRate Difference (LTR - FDDN): -28.49680299976444 l/s
EPOCH: 2 ;	Progress: 4.0% ;	MSE Flowloss (Train):  812.0678 ;	Error_loss (Train):  0.2782

MTR-Cf: 1.0621388753502394
Row ID: 11 	HoV: ['C6']
FlowRate Difference (LTR - FDDN): -17.15596913514321 l/s
EPOCH: 3 ;	Progress: 6.0% ;	MSE Flowloss (Train):  294.3273 ;	Error_loss (Train):  0.1008

MTR-Cf: 1.10

In [16]:
cf_df = pd.DataFrame(data=cf_hist_train)
losses = pd.DataFrame(data=losses_hist)

In [17]:
mseflow_last_loss = []
error_last_loss = []
for i in range(len(losses)):
    loss1 = losses.MSE_flowloss[i][-1]
    loss2 = losses.Error_loss[i][-1]
    mseflow_last_loss.append(loss1)
    error_last_loss.append(loss2)

cf_df['Final_MSELoss'] = mseflow_last_loss
cf_df['Final_ErrorLoss'] = error_last_loss

In [18]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['MSE_flowloss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained
    
cf_df.to_csv('data_output/NN_1CF_Output.csv',index=False)

In [19]:
# cf_df.sort_values(by = [MTR,'HoV'], inplace = True)
# print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
# cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

### Plot Training Losses

In [20]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set1,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,Set1,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette =  [y for x in temp_list for y in x] # ['red','blue']
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

In [21]:
hover = HoverTool(names = ['EpochPoints'],tooltips = [('Epoch', '@x'), ('Error','@y')])
hover.point_policy='snap_to_data'

**MSE FlowError Loss**

In [22]:
p1 = figure(title = 'MSE FlowLoss for TRAIN data points', width=1250)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))
    y1 = losses['MSE_flowloss'][i]    
    p1.line(x, y1, line_width=2, color=colors[i], alpha=0.8,name = 'EpochPoints',legend='MSE Flow Loss for {}'.format(labels[i]))    
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
p1.legend.click_policy="hide"
p1.add_tools(hover)
show(p1)

**CF-Only Loss**<br>
Error function plot, when `error_loss_choice = 'cf_only'` in initializing the PINN 

In [23]:
# p2 = figure(title = 'CF-Only Error (Loss) Function for TZ6 TRAIN data', width=1300)
# for i in range(num_lines):
#     x = list(range(1,len(losses['Error_loss'][i])+1))     
#     y2 = losses['Error_loss'][i]    
#     p2.line(x, y2, line_width=2, color= colors[i], alpha=0.5, name = 'EpochPoints', legend='Error Loss for {} using Fixed α = {}'.format(labels[i],learning_rate))
#     p2.circle(x, y2, line_width=2, color= colors[i], alpha=0.75, size = 8, name = 'EpochPoints', legend='Error Loss for {} using Fixed α = {}'.format(labels[i],learning_rate))
# p2.yaxis.axis_label = "Error Loss"
# p2.xaxis.axis_label = "Epochs"
# p2.legend.click_policy="hide"
# p2.xaxis.axis_label_text_font_size = '12pt'
# p2.yaxis.axis_label_text_font_size = '12pt'
# p2.xaxis.major_label_text_font_size= '12pt'
# p2.xaxis.major_label_standoff = 15
# p2.yaxis.major_label_text_font_size= '11pt'
# p2.legend.label_text_font_size = '12pt'
# p2.title.text_font_size = '12pt'
# p2.add_tools(hover)
# p2.output_backend = 'svg'
# export_svgs(p2, filename="plots/TZ6_1CF_ErrorLoss_CF_Only.svg")
# show(p2)

**Simplified FlowRate Error Loss**<br>
Error function plot, when `error_loss_choice = 'simplified'` in initializing the PINN 

In [24]:
p2 = figure(title = 'Simplified Flowrate Error (Loss) Function for TZ6 TRAIN data', width=1300)
for i in range(num_lines):
    x = list(range(1,len(losses['Error_loss'][i])+1))     
    y2 = losses['Error_loss'][i]    
    p2.line(x, y2, line_width=2, color= colors[i], alpha=0.5, name = 'EpochPoints', legend='Error Loss for {} using Fixed α = {}'.format(labels[i],learning_rate))
    p2.circle(x, y2, line_width=2, color= colors[i], alpha=0.75, size = 8, name = 'EpochPoints', legend='Error Loss for {} using Fixed α = {}'.format(labels[i],learning_rate))
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
p2.xaxis.axis_label_text_font_size = '12pt'
p2.yaxis.axis_label_text_font_size = '12pt'
p2.xaxis.major_label_text_font_size= '11pt'
p2.xaxis.major_label_standoff = 15
p2.yaxis.major_label_text_font_size= '11pt'
p2.legend.label_text_font_size = '12pt'
p2.title.text_font_size = '12pt'
p2.add_tools(hover)
p2.output_backend = 'svg'
# export_svgs(p2, filename="plots/TZ6_1CF_ErrorLoss_Simplified.svg")
show(p2)